In [8]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pymongo in c:\users\desig\appdata\local\programs\python\python313\lib\site-packages (4.15.1)




[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# import stuff


In [18]:
from os import listdir
import pandas as pd
from bson import objectid

def df_load(path):
    raw_data=listdir(path)
    cleaned = []
    dfs = {}  
    
    for f in raw_data:
        name = f.removesuffix(".csv")      # remove extension
        if "_" in name:
            name = name.split("_", 1)[1]   # take everything after the first "_"
        name = name.removeprefix("_")
        name= name.removesuffix("__anonymized")
        name= name.removesuffix("_anon")

        cleaned.append(name)
        
        
        df = pd.read_csv(f'{path}/{f}')  # adjust path if needed
        
        # store in dictionary
        dfs[name] = df
    return dfs

# unpacking script
# for name, df in dfs.items():
#     globals()[name] = df
#     print(name)

def remove_columns(df):
    for col in df.columns:
        if df[col].notnull().sum()==0:
            df.drop(columns=[col],inplace=True)

    for col in df.columns:
        if df[col].nunique()==1:
            df.drop(columns=[col],inplace=True)
    return df


def get_creation_time(x):
    if pd.isna(x):  
        return pd.NaT
    try:
        return objectid.ObjectId(str(x)).generation_time.replace(tzinfo=None)
    except Exception:
        return pd.NaT

In [19]:
pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from func import  df_load, get_creation_time, remove_columns
import Levenshtein


In [21]:
dfs=df_load('data')
for name, df in dfs.items():
    globals()[name] = df
    remove_columns(df)
    print(name)
    #print(df.columns)
    

employees
hubspot_sales_pipeline_stages
notion_clients
notion_hr
notion_roles
projects
fortnox_invoices
fortnox_supplier_invoices
hubspot_deals
time_entries
qbis__activity_time
qbis__project_activities


In [22]:
hubspot_sales_pipeline_stages

,pipeline_stage_id,pipeline_stage_order,pipeline_stage,close_probability,deal_is_closed
0,1102499,1,Qualification,0.2,False
1,12008384,0,Lead,0.1,False
2,qualifiedtobuy,3,Proposal,0.6,False
3,appointmentscheduled,2,Exploration,0.4,False
4,closedlost,5,Closed Lost,0.0,True
5,closedwon,4,Closed Won,1.0,True


In [23]:
hubspot_sales_pipeline_stages.dtypes

pipeline_stage_id        object
pipeline_stage_order      int64
pipeline_stage           object
close_probability       float64
deal_is_closed             bool
dtype: object

In [24]:
qbis__project_activities.dtypes

project_activity_id      int64
project_id               int64
activity_name           object
is_active                 bool
is_complete               bool
is_chargeable             bool
factor                 float64
dtype: object

In [25]:
qbis__activity_time.dtypes

activity_time_id      int64
employee_id           int64
activity_id           int64
activity_date        object
minutes               int64
factor_value        float64
notes_internal       object
dtype: object

In [26]:
time_entries.dtypes

dt                 object
time_entry_id      object
project_id         object
user_id            object
billable             bool
hours             float64
billable_hours    float64
dtype: object

In [27]:
fortnox_invoices

,invoice_amount_net,invoice_amount_total,customer_number,due_date,invoice_date,final_pay_date,month_name,accounting_month,accounting_year,accounting_year_date,broker,client_anon
0,52000.0,65000.0,559296,2024-10-15,2024-09-30,NaN,September,5,2024,2024-05-28,NaN,"Mccoy, Singh and Smith"
1,136500.0,170625.0,559298,2024-12-04,2024-11-04,2024-12-04,November,7,2024,2024-07-04,NaN,"Atkinson, Bishop and Cohen"
2,192000.0,240000.0,559298,2025-01-01,2024-11-30,2024-12-30,November,7,2024,2024-07-28,NaN,"Atkinson, Bishop and Cohen"
3,133500.0,166875.0,559298,2025-02-02,2024-12-31,2025-01-31,December,8,2024,2024-08-28,NaN,"Atkinson, Bishop and Cohen"
4,212800.0,266000.0,559298,2025-03-05,2025-01-31,2025-03-05,January,9,2024,2024-09-28,NaN,"Atkinson, Bishop and Cohen"
...,...,...,...,...,...,...,...,...,...,...,...,...
610,152000.0,190000.0,559290,2025-03-05,2025-01-31,2025-03-14,January,9,2024,2024-09-28,Partner,Ali-Lawrence
611,160000.0,200000.0,559290,2025-04-03,2025-02-28,2025-04-08,February,10,2024,2024-10-28,Partner,Ali-Lawrence
612,160000.0,200000.0,559290,2025-04-30,2025-03-31,2025-05-09,March,11,2024,2024-11-28,Partner,Ali-Lawrence
613,128000.0,160000.0,559290,2025-06-05,2025-04-30,2025-06-12,April,12,2024,2024-12-28,Partner,Ali-Lawrence


In [28]:
hubspot_deals.dtypes

deal_id                     int64
deal_amount               float64
deal_stage                 object
deal_close_probability    float64
create_date                object
last_modified_date         object
close_date                 object
owner_id                    int64
weighted_deal_amount      float64
deal_name_anon             object
dtype: object

In [29]:
fortnox_supplier_invoices.dtypes

invoice_payment    float64
categorization      object
invoice_date        object
due_date            object
final_pay_date      object
dtype: object

In [30]:
fortnox_invoices.dtypes

invoice_amount_net      float64
invoice_amount_total    float64
customer_number           int64
due_date                 object
invoice_date             object
final_pay_date           object
month_name               object
accounting_month          int64
accounting_year           int64
accounting_year_date     object
broker                   object
client_anon              object
dtype: object

In [31]:
employees.dtypes

employee_id       int64
employee_code     int64
first_name       object
last_name        object
practice         object
dtype: object

In [16]:
projects.dtypes

project_id                   object
client_id                    object
clockify_project_billable      bool
project_duration             object
estimated_durationn          object
client_anon                  object
project_anon                 object
dtype: object

In [15]:
notion_roles.dtypes

role_id                object
industry_name          object
role_category_name     object
hourly_rate           float64
billing_type           object
seniority              object
startdate              object
name_anon              object
dtype: object

In [13]:
notion_hr.dtypes

consultant_id           object
active                  object
startdate               object
seniority               object
consultant_name_anon    object
dtype: object

In [14]:
notion_clients.dtypes

client_id       object
category        object
industry_id     object
company_size    object
company_type    object
n_roles          int64
n_projects       int64
n_people         int64
started_at      object
name_anon       object
dtype: object

# levenshtein stuff

In [8]:
employees['consultant_name_anon']=employees['first_name']+' '+employees['last_name']

In [9]:
for i in employees['consultant_name_anon']:
    min_dist=100
    closest_name=''
    for e in notion_hr['consultant_name_anon']:    
        x= Levenshtein.distance(i,e)
        if x<min_dist:
            min_dist=x
            closest_name=e
    if min_dist<3:
        employees.loc[employees['consultant_name_anon']==i,'notion_name']=closest_name
        print(f'{i}  with {min_dist}')

Luke Rhinehart  with 0
Alexander Macedon  with 0
Rachel Lamb  with 0
Bingo Storm  with 0
Astarion Baldersson  with 1
Erroll Datasson  with 0
Bertil Grindd  with 1
ilya Altman  with 1
Dag Ström  with 0
Jana Bjork  with 1


In [10]:
for i in notion_hr['consultant_name_anon']:
    min_dist=100
    closest_name=''
    for e in employees['consultant_name_anon']:    
        x= Levenshtein.distance(i,e)
        if x<min_dist:
            min_dist=x
            closest_name=e
    if min_dist<3:
        print(f'{i}  with {min_dist}')

Bertil Grind  with 1
Alexander Macedon  with 0
Luke Rhinehart  with 0
Rachel Lamb  with 0
Astarion Baldursson  with 1
Dag Ström  with 0
Bingo Storm  with 0
Ilya Altman  with 1
Jana Björk  with 1
Erroll Datasson  with 0


# making bridge table

In [ ]:
import csv

# ====== employee info ======
# employee_id : employees.employee_id
# consultant_id : notion_hr.consultant_id
# employee_name : name to clarify
employees_bridge_table = [
    {"employee_id": 2, "consultant_id": "85808593-b0e4-4163-b8f1-a5fc90b219ee", "employee_name": "Bingo Storm"},
    {"employee_id": 3, "consultant_id": "2a554628-d7f2-475f-8872-5a7057388c36", "employee_name": "Bertil Grindd"},
    {"employee_id": 4, "consultant_id": "61948bc5-3844-4219-a605-94e22f59571a", "employee_name": "Gunnar Gunnarsson"},
    {"employee_id": 5, "consultant_id": "348c564a-f3b1-42f9-95e5-cd6a9ca66aa7", "employee_name": "Erroll Datasson"},
    {"employee_id": 6, "consultant_id": "d2458d77-5056-43b5-82c4-12543225bcd9", "employee_name": "Jana Bjork"},
    {"employee_id": 7, "consultant_id": "38eebb46-9733-484c-afd2-543d79eb80b1", "employee_name": "Sila Rofat Faircloth"},
    {"employee_id": 8, "consultant_id": "8b3bf3a7-ae48-4c80-8b37-b3bb0796aea2", "employee_name": "Gabriel Lennartsson"},
    {"employee_id": 9, "consultant_id": "afd559ec-90bf-40c9-8e96-57e3d7af66fd", "employee_name": "Astarion Baldersson"},
    {"employee_id": 10, "consultant_id": "db394d2e-2ed0-47a1-b5b9-236f864768ab", "employee_name": "Dag Ström"},
    {"employee_id": 11, "consultant_id": "40d8972b-c3ad-450e-a56d-873a8cdc4368", "employee_name": "ilya Altman"},
    {"employee_id": 12, "consultant_id": "086ef6c0-d4f2-45bb-bf64-2cb51ce8fec4", "employee_name": "Rachel Lamb"},
    {"employee_id": 20, "consultant_id": "261b42da-30b3-8088-b413-f0676fb78aa4", "employee_name": "Luke Rhinehart"},
    {"employee_id": 21, "consultant_id": "261b42da-30b3-801f-ade8-e0f9f5d087b6", "employee_name": "Alexander Macedon"},
    

]

# ====== CSV ======
csv_file = "bridge_table.csv"
with open(csv_file, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["employee_id","consultant_id","user_id","employee_name"])
    writer.writeheader()
    for emp in employees:
        writer.writerow(emp)

print(f"CSV bridge table: {csv_file}")


CSV 브리지 테이블이 생성되었습니다: bridge_table.csv


In [13]:
projects.to_csv('projects.csv',index=False)

In [ ]:
employees

In [16]:

hubspot_sales_pipeline_stages.to_csv('hubspot_sales_pipeline_stages.csv', index=False)
notion_hr.to_csv('notion_hr.csv', index=False)
notion_clients.to_csv('notion_clients.csv', index=False)
notion_roles.to_csv('notion_roles.csv', index=False)
fortnox_invoices.to_csv('fornox_invoice.csv', index=False)
fortnox_supplier_invoices.to_csv('fortnox_supplier_invoices.csv', index=False)
hubspot_deals.to_csv('hubspot_deals.csv', index=False)
time_entries.to_csv('time_entries.csv', index=False)
qbis__activity_time.to_csv('qbis__activity_time.csv', index=False)
qbis__project_activities.to_csv('qbis__project_activities.csv', index=False)


In [32]:
employees.to_csv('employees.csv', index=False)

In [ ]:
employees